In [1]:
import torch
model = torch.load('D:/software/github/GZK_Code/XAI/2022.03.23/models/2022-03-24-01-39-32full_squad.pth')

In [53]:
from datasets import load_dataset
sep_squad = load_dataset('csv', data_files={'train': './sep_squad_train.csv', 'validation': './sep_squad_validation.csv'})
squad = load_dataset('squad')

Using custom data configuration default-0ae3a52d115f18f2
Reusing dataset csv (C:\Users\GZK\.cache\huggingface\datasets\csv\default-0ae3a52d115f18f2\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/2 [00:00<?, ?it/s]

Reusing dataset squad (C:\Users\GZK\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))


def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()

    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)

    common_tokens = set(pred_tokens) & set(truth_tokens)

    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0

    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)

    return 2 * (prec * rec) / (prec + rec)


def get_gold_answers(example):
    """helper function that retrieves all possible true answers from a squad2.0 example"""

    gold_answers = [answer["text"] for answer in example.answers if answer["text"]]

    # if gold_answers doesn't exist it's because this is a negative example -
    # the only correct answer is an empty string
    if not gold_answers:
        gold_answers = [""]

    return gold_answers


"""+++++++++++++++++++++++++++++++++++"""

def predict(inputs):
    output = model(inputs)
    return output.start_logits, output.end_logits


def construct_input_ref_pair(question, text, ref_token_id, sep_token_id, cls_token_id):
    question_ids = tokenizer.encode(question, add_special_tokens=False)
    text_ids = tokenizer.encode(text, add_special_tokens=False)

    # construct input token ids
    input_ids = [cls_token_id] + question_ids + [sep_token_id] + text_ids + [sep_token_id]

    # construct reference token ids
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(question_ids) + [sep_token_id] + \
                    [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(question_ids)

def predict_qt(question, text):
    input_ids, ref_input_ids, sep_id = construct_input_ref_pair(question, text, ref_token_id, sep_token_id, cls_token_id)

    indices = input_ids[0].detach().tolist()
    all_tokens = tokenizer.convert_ids_to_tokens(indices)

    ground_truth = '13'


    start_scores, end_scores = predict(input_ids)


    return (' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1])),float(torch.max(torch.softmax(start_scores[0], dim=0))),float(torch.max(torch.softmax(end_scores[0], dim=0)))
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [4]:
squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 97608
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11844
    })
})

In [58]:
import json
ids = []
ques = []
texts = []
ids = []
ques = []
texts = []
for i in squad["validation"]:
    ids.append(i['id'])
    ques.append(i['question'])
    texts.append(i['context'])
with open("worksheets.codalab.org.txt", "r+")as f:
    org = json.load(f)
print("org : ", len(org))
print("ids : ", len(ids))

org :  11873
ids :  10570


In [80]:
import torch
model = torch.load(r"D:\software\github\GZK_Code\XAI\2022.03.23\models\2022-03-24-00-43-04full_squad.pth")

In [83]:
import os
import datetime
model = torch.load(r"D:\software\github\GZK_Code\XAI\2022.03.23\models\2022-03-24-03-32-38sep_squad.pth")
time = str(datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S'))
print(time)
answers = {}
for org_id in org:
    if org_id in ids:
        text = texts[ids.index(org_id)]
        if len(text) >=512:
            text = text[:512]
        question = ques[ids.index(org_id)]
        answer, start_scores, end_scores= predict_qt(text, question)
        print("answer", answer, "\r\n start_scores", start_scores, "\r\n end_scores", end_scores)
        answers[org_id] = answer
    elif org_id not in ids:
        print(org_id, "didn't found")
        answers[org_id] = ''
json_str = json.dumps(answers)
with open(time + 'my_answers.txt', 'w') as json_file:
    json_file.write(json_str)
    
cmd = "python evaluate-v2.0.py dev-v2.0.json " + time + 'my_answers.txt'
print(cmd)
print(os.popen(cmd).read())

2022-03-29-00-07-44
answer  
 start_scores 0.08483570069074631 
 end_scores 0.1578470766544342
answer boris tr ##akh ##ten ##bro ##t 
 start_scores 0.7442752718925476 
 end_scores 0.6704331040382385
answer us ##u 
 start_scores 0.6533545851707458 
 end_scores 0.4512552320957184
5acfa31377cf76001a685684 didn't found
answer vin ##ic ##ult ##ure and tourism 
 start_scores 0.36502552032470703 
 end_scores 0.549481213092804
5a7b054521c2de001afe9cce didn't found
answer f [SEP] what park 
 start_scores 0.19368624687194824 
 end_scores 0.36951199173927307
answer their large ph ##ary ##nx , just inside the large mouth and filling most of the sac ##like body , bears " macro ##ci ##lia " at the oral end . these fused bundles of several thousand large ci ##lia are able to " bite " off pieces of prey that are too large to swallow whole – almost always other ct ##eno ##ph ##ores . in front of the field of macro ##ci ##lia , on the mouth " lips " in some species of be ##ro ##e , is a pair of narrow s

KeyboardInterrupt: 

In [85]:
import os
import datetime
model = torch.load(r"D:\software\github\GZK_Code\XAI\2022.03.23\models\2022-03-24-03-32-38sep_squad.pth")
time = str(datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S'))
print(time)
answers = {}
for org_id in org:
    if org_id in ids:
        text = texts[ids.index(org_id)]
        if len(text) >=512:
            text = text[:512]
        question = ques[ids.index(org_id)]
        answer, start_scores, end_scores= predict_qt(text, question)
        print("answer", answer, "\r\n start_scores", start_scores, "\r\n end_scores", end_scores)
        answers[org_id] = answer
    elif org_id not in ids:
        print(org_id, "didn't found")
        answers[org_id] = ''
json_str = json.dumps(answers)
with open(time + 'my_answers.txt', 'w') as json_file:
    json_file.write(json_str)
    
cmd = "python evaluate-v2.0.py dev-v2.0.json " + time + 'my_answers.txt'
print(cmd)
print(os.popen(cmd).read())

2022-03-29-00-11-01
answer  
 start_scores 0.08483570069074631 
 end_scores 0.1578470766544342
answer boris tr ##akh ##ten ##bro ##t 
 start_scores 0.7442752718925476 
 end_scores 0.6704331040382385
answer us ##u 
 start_scores 0.6533545851707458 
 end_scores 0.4512552320957184
5acfa31377cf76001a685684 didn't found
answer vin ##ic ##ult ##ure and tourism 
 start_scores 0.36502552032470703 
 end_scores 0.549481213092804
5a7b054521c2de001afe9cce didn't found
answer f [SEP] what park 
 start_scores 0.19368624687194824 
 end_scores 0.36951199173927307
answer their large ph ##ary ##nx , just inside the large mouth and filling most of the sac ##like body , bears " macro ##ci ##lia " at the oral end . these fused bundles of several thousand large ci ##lia are able to " bite " off pieces of prey that are too large to swallow whole – almost always other ct ##eno ##ph ##ores . in front of the field of macro ##ci ##lia , on the mouth " lips " in some species of be ##ro ##e , is a pair of narrow s

KeyboardInterrupt: 

In [86]:
import os
import datetime
model = torch.load(r"D:\software\github\GZK_Code\XAI\2022.03.23\models\2022-03-24-00-43-04full_squad.pth")
time = str(datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S'))
print(time)
answers = {}
for org_id in org:
    if org_id in ids:
        text = texts[ids.index(org_id)]
        if len(text) >=512:
            text = text[:512]
        question = ques[ids.index(org_id)]
        answer, start_scores, end_scores= predict_qt(text, question)
        print("answer", answer, "\r\n start_scores", start_scores, "\r\n end_scores", end_scores)
        answers[org_id] = answer
    elif org_id not in ids:
        print(org_id, "didn't found")
        answers[org_id] = ''
json_str = json.dumps(answers)
with open(time + 'my_answers.txt', 'w') as json_file:
    json_file.write(json_str)
    
cmd = "python evaluate-v2.0.py dev-v2.0.json " + time + 'my_answers.txt'
print(cmd)
print(os.popen(cmd).read())

2022-03-29-00-11-22
answer non ##con ##ser ##vati ##ve forces other than friction include other contact forces , tension 
 start_scores 0.10350385308265686 
 end_scores 0.137172669172287
answer boris tr ##akh ##ten ##bro ##t 
 start_scores 0.9696829319000244 
 end_scores 0.9595809578895569
answer us ##u 
 start_scores 0.36553657054901123 
 end_scores 0.64606773853302
5acfa31377cf76001a685684 didn't found
answer vin ##ic ##ult ##ure and tourism 
 start_scores 0.4773162007331848 
 end_scores 0.6734422445297241
5a7b054521c2de001afe9cce didn't found
answer f [SEP] what park 
 start_scores 0.25645866990089417 
 end_scores 0.31622299551963806
answer  
 start_scores 0.19033433496952057 
 end_scores 0.13417784869670868
answer graves 
 start_scores 0.12982338666915894 
 end_scores 0.272141695022583
answer what 
 start_scores 0.40740829706192017 
 end_scores 0.6236616373062134
answer a strategy of islam ##ization of society through state power seized by revolution 
 start_scores 0.43945837020874

KeyboardInterrupt: 

In [71]:
import torch
# model = torch.load(r"D:\software\github\GZK_Code\XAI\2022.03.23\models\2022-03-24-03-32-38sep_squad.pth")
import os
import datetime
time = str(datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S'))
print(time)
answers = {}
for org_id in org:
    if org_id in ids:
        text = texts[ids.index(org_id)]
        if len(text) >=512:
            text = text[:512]
        question = ques[ids.index(org_id)]
        answer, start_scores, end_scores= predict_qt(text, question)
        answers[org_id] = answer
    elif org_id not in ids:
        #print(org_id, "didn't found")
        answers[org_id] = ''
json_str = json.dumps(answers)
with open(time + 'my_answers.txt', 'w') as json_file:
    json_file.write(json_str)
    
cmd = "python evaluate-v2.0.py dev-v2.0.json " + time + 'my_answers.txt'
print(cmd)
print(os.popen(cmd).read())

2022-03-28-23-29-03
python evaluate-v2.0.py dev-v2.0.json 2022-03-28-23-29-03my_answers.txt
{
  "exact": 53.13737050450602,
  "f1": 56.091620670370645,
  "total": 11873,
  "HasAns_exact": 6.140350877192983,
  "HasAns_f1": 12.057323248871679,
  "HasAns_total": 5928,
  "NoAns_exact": 100.0,
  "NoAns_f1": 100.0,
  "NoAns_total": 5945
}



In [88]:
import torch
import nltk as tk
model = torch.load(r"D:\software\github\GZK_Code\XAI\2022.03.23\models\2022-03-24-00-43-04full_squad.pth")
import os
import datetime
time = str(datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S'))
print(time)
answers = {}
for org_id in org:
    if org_id in ids:
        print("*"*74)
        text = texts[ids.index(org_id)]
        if len(text) >=512:
            text = text[:512]
        question = ques[ids.index(org_id)]
        tokens = tk.sent_tokenize(text)
        print("tokens->>", tokens)
        print("question", question)
        start_scores = []
        end_scores = []
        temp_answers = []
        for token in tokens:
            answer, start_score, end_score= predict_qt(text, token)
            start_scores.append(start_score)
            end_scores.append(end_score)
            temp_answers.append(answer)
        answer = temp_answers[end_scores.index(max(end_scores))]
        print("answer", temp_answers, "\r\n start_scores", start_scores, "\r\n end_scores", end_scores)
        answers[org_id] = answer
    elif org_id not in ids:
        print(org_id, "didn't found")
        answers[org_id] = ''
json_str = json.dumps(answers)
with open(time + 'my_answers.txt', 'w') as json_file:
    json_file.write(json_str)
    
cmd = "python evaluate-v2.0.py dev-v2.0.json " + time + 'my_answers.txt'
print(cmd)
print(os.popen(cmd).read())

2022-03-29-00-13-57
**************************************************************************
tokens->> ['For certain physical scenarios, it is impossible to model forces as being due to gradient of potentials.', 'This is often due to macrophysical considerations that yield forces as arising from a macroscopic statistical average of microstates.', 'For example, friction is caused by the gradients of numerous electrostatic potentials between the atoms, but manifests as a force model that is independent of any macroscale position vector.', 'Nonconservative forces other than friction include other contact forces, tension,']
question Why are some forces due to that are impossible to model?
answer ['due to gradient of potential ##s', 'a macro ##scopic statistical average of micro ##sta ##tes . for example , friction is caused by the gradient ##s of numerous electro ##static potential ##s between the atoms , but manifest ##s as a force model that is independent of any macro ##sca ##le posit

KeyboardInterrupt: 

In [87]:
import torch
import nltk as tk
model = torch.load(r"D:\software\github\GZK_Code\XAI\2022.03.23\models\2022-03-24-03-32-38sep_squad.pth")
import os
import datetime
time = str(datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S'))
print(time)
answers = {}
for org_id in org:
    if org_id in ids:
        print("*"*74)
        text = texts[ids.index(org_id)]
        if len(text) >=512:
            text = text[:512]
        question = ques[ids.index(org_id)]
        tokens = tk.sent_tokenize(text)
        print("tokens->>", tokens)
        print("question", question)
        start_scores = []
        end_scores = []
        temp_answers = []
        for token in tokens:
            answer, start_score, end_score= predict_qt(text, token)
            start_scores.append(start_score)
            end_scores.append(end_score)
            temp_answers.append(answer)
        answer = temp_answers[end_scores.index(max(end_scores))]
        print("answer", temp_answers, "\r\n start_scores", start_scores, "\r\n end_scores", end_scores)
        answers[org_id] = answer
    elif org_id not in ids:
        print(org_id, "didn't found")
        answers[org_id] = ''
json_str = json.dumps(answers)
with open(time + 'my_answers.txt', 'w') as json_file:
    json_file.write(json_str)
    
cmd = "python evaluate-v2.0.py dev-v2.0.json " + time + 'my_answers.txt'
print(cmd)
print(os.popen(cmd).read())

2022-03-29-00-13-28
**************************************************************************
tokens->> ['For certain physical scenarios, it is impossible to model forces as being due to gradient of potentials.', 'This is often due to macrophysical considerations that yield forces as arising from a macroscopic statistical average of microstates.', 'For example, friction is caused by the gradients of numerous electrostatic potentials between the atoms, but manifests as a force model that is independent of any macroscale position vector.', 'Nonconservative forces other than friction include other contact forces, tension,']
question Why are some forces due to that are impossible to model?
answer ['gradient of potential ##s', 'macro ##physical considerations that yield forces as arising from a macro ##scopic statistical average of micro ##sta ##tes', 'macro ##physical considerations that yield forces as arising from a macro ##scopic statistical average of micro ##sta ##tes', 'macro ##phys

KeyboardInterrupt: 

In [91]:
from transformers import BertTokenizer, BertForQuestionAnswering, BertConfig
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model.bert.embeddings

BertEmbeddings(
  (word_embeddings): Embedding(30522, 1024, padding_idx=0)
  (position_embeddings): Embedding(512, 1024)
  (token_type_embeddings): Embedding(2, 1024)
  (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [92]:
from transformers import BertTokenizer, BertForQuestionAnswering, BertConfig
model = torch.load(r"D:\software\github\GZK_Code\XAI\2022.03.23\models\2022-03-24-03-32-38sep_squad.pth")
model.bert.embeddings

AttributeError: 'DistilBertForQuestionAnswering' object has no attribute 'bert'